# 1. TF-IDF
1. 장점
    - 직관적인 해석이 가능함
2. 단점
    - 대규모 말뭉치를 다룰 때 메모리상의 문제가 발생
    - 높은 차원을 가짐
    - 매우 sparse한 형태의 데이터임
3. Represent Items
    - Items을 벡터 형태로 표현. 도메인에 따라 다른 방법이 적용

# 2. Word2Vec
1. 통계기반의 방법 단점
    - 대규모 말뭉치를 다룰 때 메모리 상의 문제가 발생
        - 높은 차원을 가짐, 매우 sparse한 형태의 데이터임
        - 예) 100만개의 문서를 다루는 경우 : 
        -   100만개의 문서에 등장한 모든 단어를 추출해야 하고 이때 단어의 수는 1문서당 새로운 단어가 10개면,
        -   1000만개정도의 말뭉치가 형성됨. 즉, 100만 * 1000만의 매트릭스가 형성
    - 한번에 학습 데이터 전체를 진행함
        - 큰 작업을 처리하기 어려움
        - GPU와 같은 병렬처리를 기대하기 힘들다.
    - 학습을 통해서 개선하기가 어려움
2. 통계 기반 기법 vs. 추론 기반 기법
    - 통계 기반 기법 (배치학습) : 학습데이터 -> `TF-IDF`
    - 추론 기반 기법 (미니배치 학습) : 학습데이터 -> `신경망`
3. 추론 기반의 방법 (Word2Vec)
    - 추론 : 주변 단어(**맥락**)이 주어졌을 때 "?"에 무슨 단어(**중심단어**)가 들어가는지를 추측하는 작업
    - 1) CBOW 모델 : you ___ goodbye and I say hello.
    - 2) skip-gram 모델 : ___ say ___ and I say hello.
    - 추론 기반 기법 : 맥락을 입력하면 모델은 각 단어의 출현 확률을 출력한다.
        - 예시) `{ you, goodbye} -- 맥락` => `모델` => `확률분포`
        -   모델은 {you, say, goodbye, and, I ,hello, .}이라는 말뭉치 후보군중에서 하나를 예측함
        - 잘못 예측한 경우 => 학습을 통해서 점점 모델을 개선
